In [ ]:
from torch.utils.data import Dataset, DataLoader
from motsynth import MOTSynth
import os
import json
import numpy as np
import cv2
import glob
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
data_dir = "/storage/user/brasoand/motsyn2/frames/"

In [ ]:
data = MOTSynth(data_dir)

In [ ]:
train_loader = DataLoader(
        data, batch_size=4, shuffle=True, num_workers=args.workers, drop_last=True)

In [ ]:
suitable_video_list_only_from_top = [2, 10, 19, 28, 33, 34, 42, 43, 50, 52, 59, 60, 62, 68,
                                     69, 73, 77, 78, 84, 86, 87, 90, 94, 96, 103, 104, 112, 113,
                                     121, 122, 130, 138, 147, 156, 161, 162, 170, 171, 178, 180,
                                     187, 188, 196, 197]
len(suitable_video_list_only_from_top)

In [ ]:
with open("/storage/user/brasoand/motsyn2/annotations/001.json") as fp:
    annotation = json.load(fp)

In [ ]:
segmentations = [an.get('segmentation') for an in annotation.get('annotations')]
frame_id = 10000
# frame_indices = [an.get('image_id') for an in annotation.get('annotations')]
frame_indices = np.array([an.get('image_id') for an in annotation.get('annotations')])
changed_indices = np.where(frame_indices[:-1] != frame_indices[1:])[0]

In [ ]:
frame_indices[-1]

In [ ]:
changed_indices[-4:]

In [ ]:
len(segmentations)

In [ ]:
len(annotation.get('annotations'))

In [ ]:
os.listdir()

In [ ]:
global_idx = 0

In [ ]:
def convert_coco_poly_to_mask(segmentations, height, width):
    masks = []
    for polygons in segmentations:
        if isinstance(polygons['counts'], list):
            rles = coco_mask.frPyObjects(polygons, height, width)
        
        else:
            rles = [polygons]

        mask = coco_mask.decode(rles)
        if len(mask.shape) < 3:
            mask = mask[..., None]
        #mask = torch.as_tensor(mask, dtype=torch.uint8)
        #mask = mask.any(dim=2)
        masks.append(mask)
    # if masks:
    #     masks = torch.stack(masks, dim=0)
    # else:
    #     masks = torch.zeros((0, height, width), dtype=torch.uint8)
    return masks

In [ ]:
def exclude_bbg(video_id, global_idx):
    with open(f"/storage/user/brasoand/motsyn2/annotations/{video_id}.json") as fp:
        annotation = json.load(fp)
    print("Annotation File loaded.")
    frame_src = f"/storage/user/brasoand/motsyn2/frames/{video_id}/rgb/"
    segmentations = [an.get('segmentation') for an in annotation.get('annotations')]
    
#     frame_indices = [an.get('image_id') for an in annotation.get('annotations')]
    frame_indices = np.array([an.get('image_id') for an in annotation.get('annotations')])
    changed_indices = np.where(frame_indices[:-1] != frame_indices[1:])[0]
    constant = frame_indices[0]
    prev_change = 0
    total_masks_list = []
    black_bgg_img_list = []
    for k, idx in enumerate(changed_indices):
        frame_id = frame_indices[prev_change] - constant
        print(frame_id)
        original_img =mpimg.imread(f'/storage/user/brasoand/motsyn2/frames/{video_id}/rgb/{str(frame_id).zfill(4)}.jpg')
        f_segmentations = segmentations[prev_change:(idx+1)]
        prev_change = idx+1
        masks = [convert_coco_poly_to_mask([s], 1920, 1080) for s in f_segmentations]
        total_masks = np.sum(masks, 0).squeeze()
#         total_masks_list.append(total_masks)

        masked_img = np.uint8(original_img * total_masks.reshape((1080,1920,1)))
        im = Image.fromarray(masked_img)
        im.save(f"/usr/stud/gueley/Git/SCALOR/static_bc_frames/{str(global_idx).zfill(6)}.jpeg")
        global_idx += 1
        
#         black_bgg_img_list.append(np.uint8(masked_img))
#         print(f"{k}/{len(changed_indices)} done.", end='\r')
        
    return global_idx

In [ ]:
# frame_indices = np.array([an.get('image_id') for an in annotation.get('annotations')])
# changed_indices = np.where(frame_indices[:-1] != frame_indices[1:])[0]

In [ ]:
"2".zfill(4)

In [ ]:
black_bgg_img_list, total_masks_list = exclude_bbg("001")

In [ ]:
imgplot = plt.imshow(np.uint8(black_bgg_img_list[0]))

In [ ]:
# for i in changed_indices:
#     print(i)

In [ ]:
frame_indices[changed_indices[0]+1:changed_indices[1]+1]

In [ ]:
frame_indices[52]

In [ ]:
len(frame_indices)

In [ ]:
annotation.get('annotations')[0]

In [ ]:
len(glob.glob("static_bc_frames_2/*"))

In [ ]:
data_dir = "static_bc_frames_2"
frame_dirs = sorted([os.path.join(data_dir, a) for a in os.listdir(data_dir)])

In [ ]:
# frame_dirs

In [ ]:
suitable_video_list_only_from_top = [2, 10, 19, 28, 33, 34, 42, 43, 50, 52, 59, 60, 62, 68,
                                     69, 73, 77, 78, 84, 86, 87, 90, 94, 96, 103, 104, 112, 113,
                                     121, 122, 130, 138, 147, 156, 161, 162, 170, 171, 178, 180,
                                     187, 188, 196, 197]

In [ ]:
len(suitable_video_list_only_from_top)

In [ ]:
seq_len = 10

import os
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
from common import *
from PIL import Image, ImageFile
import glob
import itertools
ImageFile.LOAD_TRUNCATED_IMAGES = True
import time
class MOTSynthBlackBG(Dataset):
    def __init__(self, data_dir, train=True):
        
        self.data_dir = data_dir
        self.skip_freq = 5
        self.phase_train = train
        
        self.frame_dirs = sorted([os.path.join(data_dir, a) for a in os.listdir(data_dir)])
        
        
        if len(self.frame_dirs) != 79200:
            raise Exception("Something is not correct.")
        
#         if self.phase_train:
#             self.frame_dirs = self.frame_dirs[:-(len(self.frame_dirs) // 10)]
#         else:
#             self.frame_dirs = self.frame_dirs[-(len(self.frame_dirs) // 10):]
            
        print("Frame Dirs constructed.", flush=True)
        self.video_id = 0
        self.idx = 0
        print("Dataset initialized.", flush=True)
        
    def get_frames(self, index, seq_len=10, skip_freq=5):
        b = np.random.randint(0,5,(1))[0]
        starting_idx = index * (seq_len * skip_freq) + np.random.randint(0,5,(1))[0]
        return np.arange(starting_idx, starting_idx + (seq_len*skip_freq), 5 )
    
    def __getitem__(self, index):
        index_list = self.get_frames(index, seq_len, self.skip_freq)
        image_list = []
        k = int(np.random.rand(1)[0]*840)
        print(index_list)
        for idx in index_list:
            f_n = self.frame_dirs[idx]
            im = Image.open(f_n)
                
            im = im.crop(box=(k, 0, 1920 - (840-k) ,1080))
            im_array = np.array(im)
            
            im = im.resize((img_h, img_w), resample=Image.BILINEAR)
            im_tensor = torch.from_numpy(np.array(im) / 255).permute(2, 0, 1)
            image_list.append(im_tensor)


        img = torch.stack(image_list, dim=0)

        return img.float(), torch.zeros(1)
    def __len__(self):
        return int(len(self.frame_dirs) / (seq_len * self.skip_freq))

In [ ]:
dataset = MOTSynthBlackBG(data_dir)

In [ ]:
len(dataset.frame_dirs)

In [ ]:
train_loader = DataLoader(
        dataset, batch_size=12, shuffle=True)

In [ ]:
x, _ = dataset.__getitem__(1583)

In [ ]:
x.shape

In [ ]:
dataset.__len__()

In [ ]:
79200/50

#### Single Video Batch DATASET

In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
from common import *
from PIL import Image, ImageFile
import glob
import itertools
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask
top_down_list = [19, 28, 33, 42, 59, 60, 68, 73, 86,
                 94, 103, 112, 121, 130, 147, 161, 
                 170, 178, 187]
top_down_list = [str(a).zfill(3) for a in top_down_list]

class MOTSynthV2(Dataset):
    def __init__(self, data_dir, train=False):

        self.data_dir = data_dir
        self.skip_freq = 5
        self.phase_train = train
        self.current_video = None
        #         self.video_dirs = [os.path.join(data_dir, video_dir) for video_dir in os.listdir(data_dir)]
        
        print(f"Dataset Length: {len(top_down_list)}")
        self.video_dirs = [
            os.path.join(data_dir, video_dir) for video_dir in top_down_list
        ]

        random.shuffle(self.video_dirs)
        
        # if self.phase_train:
        #     self.video_dirs = self.video_dirs[: -(len(self.video_dirs) // 10)]
        # else:
        #     self.video_dirs = self.video_dirs[-(len(self.video_dirs) // 10) :]
        print(f"DataSet Used: MOTSynth", flush=True)
        print(f"Suitable Videos: {self.video_dirs}", flush=True)
        print(f"Suitable Videos Length: {len(self.video_dirs)}", flush=True)
        print("Video Dirs constructed.", flush=True)
        self.frame_dirs = [
            sorted(glob.glob(video_dir + "/rgb/*")) for video_dir in self.video_dirs
        ]
        self.frame_dirs = sorted(list(itertools.chain(*self.frame_dirs)))
        print("Frame Dirs constructed.", flush=True)
        #         self.frame_dirs = []
        self.video_id = 0
        self.idx = 0
        print("Dataset initialized.", flush=True)

    def get_frames(self, index, seq_len=10, skip_freq=5):
        b = np.random.randint(0, 5, (1))[0]
        starting_idx = index * (seq_len * skip_freq) + np.random.randint(0, 5, (1))[0]
        return np.arange(starting_idx, starting_idx + (seq_len * skip_freq), 5)

    def __getitem__(self, index):
        #         print(index)
        current_video_frame_dirs = [f for f in self.frame_dirs if f"frames/{self.current_video}" in f]
#         print((current_video_frame_dirs[0:5]))
        real_index = index % 36
        index_list = self.get_frames(real_index, seq_len, self.skip_freq)
        image_list = []
        #         print("-------------------------", flush=True)
        k = int(np.random.rand(1)[0] * 840)
        for idx in index_list:
#             f_n = self.frame_dirs[idx]
            f_n = current_video_frame_dirs[idx]
            im = Image.open(f_n)

            im = im.crop(box=(k, 0, 1920 - (840 - k), 1080))
            im_array = np.array(im)

            im = im.resize((img_h, img_w), resample=Image.BILINEAR)
            im_tensor = torch.from_numpy(np.array(im) / 255).permute(2, 0, 1)
            image_list.append(im_tensor)


        img = torch.stack(image_list, dim=0)
        print(index)
        return img.float(), self.current_video

    def __len__(self):
        return int(len(self.frame_dirs) / (seq_len * self.skip_freq))

In [ ]:
data_dir = "/storage/user/brasoand/motsyn2/frames/"
data = MOTSynthV2(data_dir)
data.current_video = random.choice(top_down_list)
train_loader = DataLoader(
        data, batch_size=1, shuffle=True, num_workers=0, drop_last=True)

In [ ]:
data.current_video = random.choice(top_down_list)
train_loader.dataset.current_video

In [ ]:
# a= data.__getitem__(4)
# data.current_video = random.choice(top_down_list)
# a = data.__getitem__(4)
# data.current_video = random.choice(top_down_list)
# a = data.__getitem__(4)
data.current_video = random.choice(top_down_list)
generator = iter(train_loader)
sample, counting_gt = next(generator)
print(counting_gt)

# generator._dataset.current_video = random.choice(top_down_list)
data.current_video = random.choice(top_down_list)

sample, counting_gt = next(generator)
print(counting_gt)
# generator._dataset.current_video = random.choice(top_down_list)
data.current_video = random.choice(top_down_list)

sample, counting_gt = next(generator)
print(counting_gt)

In [ ]:
data.current_video = random.choice(top_down_list)

sample, counting_gt = next(generator)

In [ ]:
dir(generator)

In [ ]:
train_loader.dataset.current_video = random.choice(top_down_list)
sample, counting_gt = next(generator)

In [ ]:
counting_gt

In [ ]:
train_loader.dataset.current_video

In [ ]:
random.sample(top_down_list, len(top_down_list))

In [ ]:
top_down_list

In [ ]:
data.current_video = random.choice(top_down_list)
sample, counting_gt = next(generator)

In [ ]:
# video_id_embs

In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
from common import *
from PIL import Image, ImageFile
import glob
import itertools
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask

In [2]:
video_id_embs = {str(k):torch.randn(10) for k in top_down_list}
class MOTSynth(Dataset):
    def __init__(self, data_dir, train=False):

        self.data_dir = data_dir
        self.skip_freq = 5
        self.phase_train = train
        #         self.video_dirs = [os.path.join(data_dir, video_dir) for video_dir in os.listdir(data_dir)]

        self.video_dirs = [
            os.path.join(data_dir, video_dir) for video_dir in top_down_list
        ]

        random.shuffle(self.video_dirs)
        
        # if self.phase_train:
        #     self.video_dirs = self.video_dirs[: -(len(self.video_dirs) // 10)]
        # else:
        #     self.video_dirs = self.video_dirs[-(len(self.video_dirs) // 10) :]
        print(f"DataSet Used: MOTSynth", flush=True)
        print(f"Suitable Videos: {self.video_dirs}", flush=True)
        print(f"Suitable Videos Length: {len(self.video_dirs)}", flush=True)
        print("Video Dirs constructed.", flush=True)
        self.frame_dirs = [
            sorted(glob.glob(video_dir + "/rgb/*")) for video_dir in self.video_dirs
        ]
        self.frame_dirs = sorted(list(itertools.chain(*self.frame_dirs)))
        print("Frame Dirs constructed.", flush=True)
        #         self.frame_dirs = []
        self.video_id = 0
        self.idx = 0
        print("Dataset initialized.", flush=True)

    def get_frames(self, index, seq_len=10, skip_freq=5):
        b = np.random.randint(0, 5, (1))[0]
        starting_idx = index * (seq_len * skip_freq) + np.random.randint(0, 5, (1))[0]
        return np.arange(starting_idx, starting_idx + (seq_len * skip_freq), 5)

    def __getitem__(self, index):
        #         print(index)
        index_list = self.get_frames(index, seq_len, self.skip_freq)
        image_list = []
        #         print("-------------------------", flush=True)
        k = int(np.random.rand(1)[0] * 840)
#         k = 0
        for idx in index_list:
            f_n = self.frame_dirs[idx]
            #             print(f_n, flush=True)
            im = Image.open(f_n)

            im = im.crop(box=(k, 0, 1920 - (840 - k), 1080))
            #             print(im.size)
            im_array = np.array(im)

            #             im = im.crop(box=(left_edge, upper_edge, left_edge + self.args.train_station_cropping_origin,
            #                               upper_edge + self.args.train_station_cropping_origin))
            im = im.resize((img_h, img_w), resample=Image.BILINEAR)
            im_tensor = torch.from_numpy(np.array(im) / 255).permute(2, 0, 1)
            image_list.append(im_tensor)
        #         print("-------------------------", flush=True)

        current_video_id = f_n.split("/")[-3]
        print(current_video_id)
        img = torch.stack(image_list, dim=0)
        #         print(img.shape)
        #         self.idx +=1
        #         print(f"Status: {self.idx}/{int(len(self.frame_dirs) / seq_len)} done.", end='\r', flush=True)
        return img.float(), video_id_embs[current_video_id]

    def __len__(self):
        return int(len(self.frame_dirs) / (seq_len * self.skip_freq))

In [ ]:
data_dir = "/storage/user/brasoand/motsyn2/frames/"
data = MOTSynth(data_dir)
data.current_video = random.choice(top_down_list)
train_loader = DataLoader(
        data, batch_size=10, shuffle=True, num_workers=0, drop_last=True)

In [ ]:
generator = iter(train_loader)
sample, counting_gt = next(generator)

In [ ]:
counting_gt.shape

In [ ]:
top_down_list

In [ ]:
counting_gt 

In [8]:
{str(k):torch.rand(10) for k in top_down_list}

{'028': tensor([0.5698, 0.4523, 0.0439, 0.2748, 0.6137, 0.7246, 0.6272, 0.7679, 0.0314,
         0.7192]),
 '033': tensor([0.3194, 0.7520, 0.0468, 0.1089, 0.3940, 0.0794, 0.8959, 0.4173, 0.8779,
         0.1524]),
 '042': tensor([0.7498, 0.8630, 0.4793, 0.3743, 0.9638, 0.0117, 0.2004, 0.5346, 0.1427,
         0.7409]),
 '059': tensor([0.4577, 0.9213, 0.9728, 0.2543, 0.1908, 0.1222, 0.6117, 0.9399, 0.3682,
         0.0634]),
 '060': tensor([0.0416, 0.6504, 0.6510, 0.1798, 0.1896, 0.4336, 0.1933, 0.5292, 0.7736,
         0.6729]),
 '068': tensor([0.3736, 0.4585, 0.8045, 0.1315, 0.6613, 0.8539, 0.3696, 0.3114, 0.0410,
         0.8416]),
 '073': tensor([0.0547, 0.0443, 0.4126, 0.3588, 0.4887, 0.5848, 0.3670, 0.9690, 0.9729,
         0.0097]),
 '086': tensor([0.0956, 0.6294, 0.2731, 0.4732, 0.8368, 0.2797, 0.9791, 0.6993, 0.5390,
         0.4438])}

In [ ]:
torch.tensor()

In [5]:
{'028': torch.tensor([0.5698, 0.4523, 0.0439, 0.2748, 0.6137, 0.7246, 0.6272, 0.7679, 0.0314,
         0.7192]),
 '033': torch.tensor([0.3194, 0.7520, 0.0468, 0.1089, 0.3940, 0.0794, 0.8959, 0.4173, 0.8779,
         0.1524]),
 '042': torch.tensor([0.7498, 0.8630, 0.4793, 0.3743, 0.9638, 0.0117, 0.2004, 0.5346, 0.1427,
         0.7409]),
 '059': torch.tensor([0.4577, 0.9213, 0.9728, 0.2543, 0.1908, 0.1222, 0.6117, 0.9399, 0.3682,
         0.0634]),
 '060': torch.tensor([0.0416, 0.6504, 0.6510, 0.1798, 0.1896, 0.4336, 0.1933, 0.5292, 0.7736,
         0.6729]),
 '068': torch.tensor([0.3736, 0.4585, 0.8045, 0.1315, 0.6613, 0.8539, 0.3696, 0.3114, 0.0410,
         0.8416]),
 '073': torch.tensor([0.0547, 0.0443, 0.4126, 0.3588, 0.4887, 0.5848, 0.3670, 0.9690, 0.9729,
         0.0097]),
 '086': torch.tensor([0.0956, 0.6294, 0.2731, 0.4732, 0.8368, 0.2797, 0.9791, 0.6993, 0.5390,
         0.4438])}

{'028': tensor([-0.9431,  0.3764, -0.1398, -0.6531,  0.0526, -0.3063, -0.5410, -1.9295,
          0.1432, -2.3675]),
 '033': tensor([-0.0989,  0.6890,  1.2964,  1.3719,  1.8907,  0.0976, -0.6488, -1.6996,
         -0.3500,  0.9189]),
 '042': tensor([ 0.6292, -0.7657, -1.6421, -0.5241, -1.0109,  0.2165, -0.7254,  1.7617,
          0.3518,  0.9029]),
 '059': tensor([-0.8495, -0.7542,  0.8164,  0.6392,  2.3996,  0.7730, -0.3945, -1.1212,
          0.6336, -0.9527]),
 '060': tensor([ 0.7131,  0.4176, -0.6837,  1.5498, -1.0839, -0.6990,  0.3572, -0.7721,
          0.9399,  0.6562]),
 '068': tensor([-0.0179, -1.2116,  0.6126,  0.4204,  0.9386, -0.8316, -0.5078, -0.2547,
          0.9024, -0.6797]),
 '073': tensor([ 0.5031, -0.6749, -0.7594, -0.2348,  0.1629, -0.4459,  0.9873,  0.3375,
         -1.0550, -0.2310]),
 '086': tensor([ 0.4655,  0.6293,  1.0465, -1.1338, -1.1782,  0.2136, -0.8174, -0.1605,
         -0.7459, -0.4709])}